### Erstellen des Suchindex

Die Keyword Search ist in Kedro angelegt. Um etwa eine query auf der Test-Doku auszuführen, muss zunächst der Suchindex auf der Doku erstellt werden. Dafür kann einfach die pre_processing pipeline ausgeführt werden. Sie enthält nodes, die sowohl die Doku parsen, als auch im nächsten Schritt einen Suchindex auf den geparsten Paragraphen erstellt.

In [1]:
session.run(pipeline_name='dp')

2022-05-30 14:23:22,938 - kedro.framework.session.session - INFO - ** Kedro project QuaselDoku


/mnt/Data/Studium/tracetronic/QuaselDoku/src/quaseldoku/pipelines/data_pre_processing/nodes.py:79: DeprecationWarning: invalid escape sequence \W
  clean_text = re.sub('\W+', ' ', clean_text)


2022-05-30 14:23:23,659 - kedro.io.data_catalog - INFO - Loading data from `ecu_test_doku` (PartitionedDataSet)...
2022-05-30 14:23:23,769 - kedro.io.data_catalog - INFO - Loading data from `params:preprocessing` (MemoryDataSet)...
2022-05-30 14:23:23,770 - kedro.pipeline.node - INFO - Running node: filter_and_flatten_doku: filter_doku([ecu_test_doku,params:preprocessing]) -> [ecu_test_doku_flat_only_html]


/home/martin/miniconda3/envs/quaseldoku/lib/python3.8/site-packages/kedro/framework/context/context.py:333: UserWarning: Credentials not found in your Kedro project config.
No files found in ['/mnt/Data/Studium/tracetronic/QuaselDoku/conf/base', '/mnt/Data/Studium/tracetronic/QuaselDoku/conf/local'] matching the glob pattern(s): ['credentials*', 'credentials*/**', '**/credentials*']
  warn(f"Credentials not found in your Kedro project config.\n{str(exc)}")


filtering out relevant html files from doku ...


...

2022-05-30 14:23:24,016 - kedro.io.data_catalog - INFO - Saving data to `germanquad_validation` (CSVDataSet)...
2022-05-30 14:23:24,150 - kedro.runner.sequential_runner - INFO - Completed 2 out of 6 tasks
2022-05-30 14:23:24,151 - kedro.io.data_catalog - INFO - Loading data from `ecu_test_doku_flat_only_html` (PartitionedDataSet)...
2022-05-30 14:23:24,177 - kedro.io.data_catalog - INFO - Loading data from `params:preprocessing` (MemoryDataSet)...
2022-05-30 14:23:24,179 - kedro.pipeline.node - INFO - Running node: parse_html_doku_to_csv: parse_html_and_combine([ecu_test_doku_flat_only_html,params:preprocessing]) -> [ecu_test_doku_parsed]
parsing html files ...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 227/227 [00:03<00:00, 69.93it/s]

2022-05-30 14:23:27,429 - kedro.io.data_catalog - INFO - Saving data to `ecu_test_doku_parsed` (CSVDataSet)...


2022-05-30 14:23:27,463 - kedro.runner.sequential_runner - INFO - Completed 3 out of 6 tasks
2022-05-30 14:23:27,464 - kedro.io.data_catalog - INFO - Loading data from `germanquad_validation` (CSVDataSet)...
2022-05-30 14:23:27,501 - kedro.io.data_catalog - INFO - Loading data from `params:germanquad_search_index` (MemoryDataSet)...
2022-05-30 14:23:27,502 - kedro.pipeline.node - INFO - Running node: index_germanquad: create_document_index([germanquad_validation,params:germanquad_search_index]) -> None
document index will be strored to: /mnt/Data/Studium/tracetronic/QuaselDoku/data/03_primary/search_indices/germanquad/
2022-05-30 14:23:31,531 - kedro.runner.sequential_runner - INFO - Completed 4 out of 6 tasks
2022-05-30 14:23:31,532 - kedro.io.data_catalog - INFO - Loading data from `ecu_test_doku_parsed` (CSVDataSet)...
2022-05-30 14:23:31,556 - kedro.pipeline.node - INFO - Running node: parse_paragraphs: blocks_to_paragraphs([ecu_test_doku_parsed]) -> [paragraph_elements]
2022-05-30

{}

### Anfrage auf dem Suchindex

Um eine Suchanfrage auf einem Suchindex zu erstellen kann dann einfach die Funktion 'keyword_search.query' verwendet werden. Neben der Anfrage (Query) muss lediglich der Pfad zum Suchindex angegeben werden. Über den Parameter 'return_context' kann angegeben werden ob nur der Hash der gefundenen Kontexte oder der Kontext selbst zurückgegeben werden soll.

In [1]:
from quaseldoku.qa_methods import keyword_search

In [2]:
docu_search_index = '../data/03_primary/search_indices/ecu_test/' 

In [6]:
res = keyword_search.query("Was ist ein Traceschritt?", docu_search_index, return_context=True)

In [7]:
res.keys()

dict_keys(['hits', 'query', 'parsed_query'])

In [12]:
print('question was:', res['query'])
print('question was parsed at:', res['parsed_query'])
print(f"search yielded {len(res['hits'])} results")
print('best result is:')
print(res['hits'][0][1][:100], '...')
print('with hash:', res['hits'][0][0])

question was: Was ist ein Traceschritt?
question was parsed at: traceschritt
search yielded 33 results
best result is:
Der Testschritt Traceschrittergebnis übernehmen ermöglicht es, das Reportergebnis eines beliebigen T ...
with hash: swBoqU/Aj0KJ1qKrXNgXrA==
